## Evaluating GPT Models

## Environmental Setup

In [ ]:
!pip install openai

If you run this file locally, make sure to save openai api key in `.env` file.

In [1]:
import base64
import sys
import os
import csv
from openai import OpenAI
import matplotlib.pyplot as plt
import pandas as pd

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    from google.colab import userdata
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
else:
    import os
    from dotenv import load_dotenv

    load_dotenv()
    OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

client = OpenAI(api_key=OPENAI_API_KEY)

In [2]:
# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

## Batch Evaluation

Define functions for batch evaluation.

In [13]:
# Function to create a request for each image
def process_images_in_batch(image_folder, prompt, client, verbose=True):
    image_files = [f for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]
    responses = []
    
    for image_file in image_files:        
        image_path = os.path.join(image_folder, image_file)
        base64_image = encode_image(image_path)
        
        # temporary code to skip first 10 images
        index = int(image_file.split('_')[-1].split('.')[0])
        if index < 10:
            continue
        
        try:
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {
                        "role": "user",
                        "content": [
                            {
                                "type": "text",
                                "text": prompt,                                
                            },
                            {
                                "type": "image_url",
                                "image_url": {
                                    "url": f"data:image/jpeg;base64,{base64_image}"
                                },
                            },
                        ],
                    }
                ],
            )
            
        except Exception as e:
            print(f"Error processing image {image_file}: {e}")
            continue
        
        if verbose:
            print(f"Filename: {image_file}| Response: {response.choices[0].message.content}")
            
        responses.append({"image_file": image_file,"response": response.choices[0].message.content})
    
    return responses

def save_responses(responses, image_folder, csv_file_path, first_write=True, verbose=False):          
    ### The file needs to exist if it's not the first write 
    if not first_write and not os.path.exists(csv_file_path):
        print("Please provide a valid CSV file path.")
        return
    
    ### determining original_country and synthesized_race automatically from the folder name
    possible_countries = ['Korea', 'UK', 'Myanmar', 'Azerbaijan']
    possible_synthesized_races = ['Asian', 'Indian', 'Black', 'White', 'Caucasian']

    original_country = [country for country in possible_countries if country in image_folder][0]

    if 'original' in image_folder:
        synthesized_race = original_country
    else:
        synthesized_race = [race for race in possible_synthesized_races if race in image_folder][0]
    
    ### saving into the csv file
    with open(csv_file_path, mode='a', newline='', encoding='utf-8') as csvfile:
        csv_writer = csv.writer(csvfile)
        
        # Write the header if it's the first write
        if first_write:
            csv_writer.writerow(['original_country', 'synthesized_race', 'image_file_name', 'response'])

        # Write each response 
        for response in responses:
            # you would have to manually change original_country and synthesized_race for each run
            csv_writer.writerow([original_country, synthesized_race, response['image_file'], response['response']])
            
            if verbose: print(f"Filename: {response['image_file']} | Response: {response['response']}")
            
        print(f"Data saved to: {csv_file_path}")
        
    ### sort the rows in the .csv file by file index
    df = pd.read_csv(csv_file_path)
    df["index"] = df["image_file_name"].apply(lambda file_name: int(file_name.split("_")[-1].split(".")[0]))
    df_sorted = df.sort_values(by=["synthesized_race", "index"], ascending=[True, True])
    df_sorted = df_sorted.drop(columns=["index"])
    df_sorted.to_csv(csv_file_path, index=False)

Evaluate all images inside a given directory and save the responses in a given `.csv` file.
- Set `first_write=True` if writing responses into a new file and needing the **header row** like `'original_country', 'synthesized_race', 'image_file_name', 'response'`.
- The function will automatically figure out `original_country` and `synthesized_race` from `image_folder`, so ensure the folder path name is meaningful.

In [ ]:
image_folder = "images/synthesized_images/clothes/Myanmar/Indian"
prompt = "Which country is the clothing in the photo mostly associated with? Which visual cues did you use to determine it?"
responses = process_images_in_batch(image_folder, prompt, client)

csv_file_path = "responses/Myanmar_Indian_Clothes_Results.csv"
save_responses(responses, image_folder, csv_file_path, first_write=True)

### Individual Evaluation

In [ ]:
# Path to your image
image_path = "images/synthesized_images/food/Myanmar/Asian/Myanmar_Asian_food_5.png"

# Getting the base64 string
base64_image = encode_image(image_path)

response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "Which country is the food in the photo mostly associated with? What is this food called?",
        },
        {
          "type": "image_url",
          "image_url": {
            "url":  f"data:image/jpeg;base64,{base64_image}"
          },
        },
      ],
    }
  ],
)


image = plt.imread(image_path)
plt.imshow(image)
plt.axis('off') 
plt.show()

print(f"Filename: {os.path.basename(image_path)}")
print(response.choices[0].message.content)
print("=" * 50)

# Utility Functions (for organizing data)


## Add Food Label Column

Referring to the country name and image index from the **csv file of food labels**, it will assign a matching label to each image file in a given **csv file of responses**. Therefore, it's important for an original image and their corresponding synthesized images to have the same index.

- For eg. Myanmar,Asian,Myanmar_Asian_food_1.png,"The food in the photo is...` will refer to `Myanmar_food_1.png,"Burmese Curry"` from `food_label.csv`.

In [23]:
import pandas as pd

def match_food_label(food_label_csv, responses_csv):
    labels_df = pd.read_csv(food_label_csv)
    details_df = pd.read_csv(responses_csv)

    # Extract the country and image number from the image_file in labels_df
    labels_df["country"] = labels_df["image_file"].str.extract(r"^([A-Za-z]+)_")
    labels_df["image_number"] = labels_df["image_file"].str.extract(r"(\d+)")

    # Extract the country and image number from the image_file_name in details_df
    details_df["country"] = details_df["image_file_name"].str.extract(r"^([A-Za-z]+)_")
    details_df["image_number"] = details_df["image_file_name"].str.extract(r"(\d+)")

    # Merge on both 'country' and 'image_number'
    merged_df = pd.merge(details_df, labels_df, on=["country", "image_number"], how="left")

    # Drop auxiliary columns
    merged_df.drop(columns=["country", "image_number"], inplace=True)

    # Reorder columns to place 'label' right after 'image_file_name'
    columns_order = ["original_country", "synthesized_race", "image_file_name", "label", "response"]
    merged_df = merged_df[columns_order]

    # Save to a new CSV file
    merged_df.to_csv(responses_csv, index=False)

food_label_csv = "images/food_label.csv"
responses_csv = "responses/gpt4o/Myanmar_Original_Food_Results.csv"
match_food_label(food_label_csv, responses_csv)

### Decrement index of images in csv file

In [ ]:
import pandas as pd

def decrement_image_index(csv_file_path):
    df = pd.read_csv(csv_file_path)
    
    # Function to decrement the index in the image file name
    def decrement_index(file_name):
        parts = file_name.split('_')
        index_part = parts[-1].split('.')[0]
        new_index = str(int(index_part) - 1)
        parts[-1] = new_index + '.' + parts[-1].split('.')[1]
        return '_'.join(parts)
    
    # Apply the function to the image file name column
    df['image_file_name'] = df['image_file_name'].apply(decrement_index)
    
    # Save the updated dataframe back to the CSV file
    df.to_csv(csv_file_path, index=False)

csv_file_path = "images/food_label.csv"
decrement_image_index(csv_file_path)

### Decrement index of images in folder

In [13]:
import os

def decrement_image_index_in_folder(folder_path):
    temp_suffix = "_temp"
    temp_files = []

    # Pass 1: Rename to temporary files to avoid overwrites
    for filename in os.listdir(folder_path):
        if filename.endswith(('.png', '.jpg', '.jpeg', '.PNG')):
            parts = filename.split('_')
            index_part = parts[-1].split('.')[0]
            
            new_index = str(int(index_part) - 1)
            parts[-1] = new_index + '.' + parts[-1].split('.')[1]
            new_filename = '_'.join(parts)
            temp_filename = new_filename + temp_suffix
            os.rename(os.path.join(folder_path, filename), os.path.join(folder_path, temp_filename))
            temp_files.append(temp_filename)


    # Pass 2: Rename from temporary to final names
    for temp_filename in temp_files:
        final_filename = temp_filename.replace(temp_suffix, "")
        os.rename(os.path.join(folder_path, temp_filename), os.path.join(folder_path, final_filename))

folder_path = r"D:\Data Downloads\Relevant\Korean_Food\synthesized_images\Black"
decrement_image_index_in_folder(folder_path)
